# Model 2.1 - Relate Review to Satisfaction

Goal 2.1 – Predict Satisfaction from Review Text

In this version, we build a deep learning classification model to predict patient satisfaction ratings (1–5) based on free-text drug reviews from the WebMD dataset.

Unlike transformer-based models, this approach uses custom-trained Word2Vec embeddings to represent patient reviews, followed by an LSTM neural network to capture the sequential structure of the text. This technique enables us to model semantic meaning and sentiment flow over time within reviews.

Key steps include:

- Preprocessing reviews (lowercasing, lemmatization, stopword removal, special character stripping)
- Training a Word2Vec model on the cleaned corpus
- Transforming reviews into padded sequences of embeddings
- Training an LSTM classifier to predict satisfaction levels

This model is applied across all drugs in the dataset, providing a scalable and interpretable alternative to large transformer-based methods, while still capturing meaningful patterns in patient language.

# Load and Preprocess the Data

## Loading CVS

In [ ]:
import pandas as pd

# Load CSV using your path
df = pd.read_csv('/Users/homecomputer/code/ElbediwiM/drug-analysis-review/Raw_Data/webmd.csv')

# Clean and prepare relevant columns
df = df[['Drug', 'Reviews', 'Satisfaction']].dropna()
df['Satisfaction'] = df['Satisfaction'].astype(int)

df.head()


,Drug,Reviews,Satisfaction
0,25dph-7.5peh,I'm a retired physician and of all the meds I ...,5
1,25dph-7.5peh,cleared me right up even with my throat hurtin...,5
2,warfarin (bulk) 100 % powder,why did my PTINR go from a normal of 2.5 to ov...,3
3,warfarin (bulk) 100 % powder,FALLING AND DON'T REALISE IT,1
4,warfarin (bulk) 100 % powder,My grandfather was prescribed this medication ...,1
